# SofaScore Web-Scrapper

### In order to Web-Scrape, you need to add the SofaScore Game ID which is in the URL of a game: for example https://www.sofascore.com/en-us/arsenal-crystal-palace/hsR#id:10385741 is the URL for the first game of the Premier Leage 22/23 season. See the Game ID at the end (10385741)? That is all you need, then you just type in the name of the Home team and Away team and the date. My code should do the rest for you! Enjoy.

In [5]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
# All you need is to input the 4 things below!
game_dict = {10385741: ['Arsenal', 'Crystal Palace', '8/5/22']}

combined_df = pd.DataFrame()

for each_id, team_info in game_dict.items():
    home_team_name, away_team_name, the_date = team_info

    response = requests.get('https://www.sofascore.com', headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'})

    soup = BeautifulSoup(response.text, 'html.parser')

    headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'if-none-match': 'W/"c5ccb17412"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',}

    response = requests.get(f'https://api.sofascore.com/api/v1/event/{each_id}/lineups', headers=headers)
    response
    headers['If-Modified-Since'] = ''

    response = requests.get(f'https://api.sofascore.com/api/v1/event/{each_id}/lineups', headers=headers)

    lineups = response.json()
    lineups_data_home = lineups['home']['players']
    confirmed_value_home = lineups['home'].get('confirmed', None)
    flattened_data_home = [{'team': home_team_name, 'confirmed': confirmed_value_home, **player['player'], **player} for player in lineups_data_home]

    lineups_data_away = lineups['away']['players']
    confirmed_value_away = lineups['away'].get('confirmed', None)
    flattened_data_away = [{'team': away_team_name, 'confirmed': confirmed_value_away, **player['player'], **player} for player in lineups_data_away]


    flattened_data = flattened_data_home + flattened_data_away


    df = pd.json_normalize(flattened_data)

    desired_columns = [
    'statistics.rating', 'team', 'shortName', 'position', 'shirtNumber',
    'substitute', 'country.name', 'statistics.minutesPlayed', 'statistics.goals', 'statistics.expectedGoals', 'statistics.goalAssist',
    'statistics.expectedAssists', 'statistics.touches', 'statistics.totalPass', 'statistics.accuratePass',
    'statistics.keyPass', 'statistics.totalCross', 'statistics.accurateCross',
    'statistics.totalLongBalls', 'statistics.accurateLongBalls', 'statistics.bigChanceCreated', 'statistics.onTargetScoringAttempt',
    'statistics.shotOffTarget', 'statistics.hitWoodwork', 'statistics.outfielderBlock', 'statistics.totalContest', 'statistics.wonContest',
    'statistics.bigChanceMissed', 'statistics.duelLost', 'statistics.duelWon', 'statistics.aerialLost', 'statistics.aerialWon',
    'statistics.possessionLostCtrl', 'statistics.fouls', 'statistics.wasFouled', 'statistics.totalOffside',
    'statistics.totalClearance', 'statistics.clearanceOffLine', 'statistics.blockedScoringAttempt', 'statistics.interceptionWon',
    'statistics.totalTackle', 'statistics.ownGoals', 'statistics.challengeLost', 'statistics.dispossessed',
    'statistics.saves', 'statistics.goalsPrevented', 'statistics.savedShotsFromInsideTheBox', 'statistics.punches', 'statistics.goodHighClaim'
     ]

    for column in desired_columns:
        if column not in df.columns:
            df[column] = None
    df1 = df[desired_columns].copy()

    df1.columns = [col.replace("statistics.", "").capitalize() if col.startswith("statistics.") else col for col in df1.columns]

    df1.rename(columns={'shortName': 'Name',
                        'position': 'Position',
                        'country.name': 'Country',
                        'Totalpass': 'TotalPasses',
                        'Accuratepass': 'AccuratePasses',
                        'Totallongballs': 'TotalLongBalls',
                        'Accuratelongballs': 'AccurateLongBalls',
                        'Totalclearance': 'TotalClearances',
                        'Totalclearance': 'TotalClearances',
                        'Goodhighclaim': 'HighClaims',
                        'Savedshotsfrominsidethebox': 'SavesInsideBox',
                        'Saves': 'SavesOutsideBox',
                        'Possessionlostctrl': 'LostPossession',
                        'Keypass': 'Keypasses',
                        'Goalsprevented': 'GoalsPrevented',
                        'Aeriallost': 'TotalAerialDuels',
                        'Aerialwon': 'AerialDuelsWon',
                        'Duellost': 'TotalGroundDuels',
                        'Duelwon': 'GroundDuelsWon',
                        'Blockedscoringattempt': 'ShotsBlocked',
                        'Ontargetscoringattempt': 'ShotOnTarget',
                        'Shotofftarget': 'ShotOffTarget',
                        'Totaloffside': 'Offsides',
                        'Goalassist' : 'Assists',
                        'Bigchancecreated': 'BigChancesCreated',
                        'Bigchancemissed': 'BigChancesMissed',
                        'Totalcontest': 'TotalDribbles',
                        'Woncontest': 'SuccessfulDribbles',
                        'ownGoals': 'OwnGoals',
                        'challengeLost': 'ChallengesLost',
                        'disspossessed': 'Disspossessed',
                        'hitWoodwork': 'HitWoodwork',
                        'outfielderBlock': 'BlockedShots',
                        }, inplace=True)


    df1.insert(0, 'Date', pd.to_datetime(the_date).date())
    df1.dropna(subset=['Rating'], inplace=True)
    df1.fillna(0, inplace=True)

    combined_df = pd.concat([combined_df, df1], ignore_index=True)

In [6]:
import pandas as pd
combined_df.to_excel('ArsenalvsPalace.xlsx', index=False)
from IPython.display import FileLink
FileLink(r'ArsenalvsPalace.xlsx')

/Users/adamcohen/Downloads/ArsenalvsPalace.xlsx